In [1]:
import numpy as np
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
onj = SentimentIntensityAnalyzer()
from textblob import TextBlob
import re
import chardet
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
%pip install scikit-learn


In [3]:
%pip install chardet

In [4]:
%pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [193]:
subreddit = [
    "gameofthrones",
    "aww",
    "gaming",
    "news",
    "politics",
    "dankmemes",
    "relationship_advice",
    "nba",
    "worldnews",
    "AskReddit",
    "AmItheAsshole",
    "SquaredCircle",
    "The_Donald",
    "leagueoflegends",
    "hockey",
    "videos",
    "teenagers",
    "gonewild",
    "movies",
    "funny",
    "pics",
    "marvelstudios",
    "memes",
    "soccer",
    "freefolk",
    "MortalKombat",
    "todayilearned",
    "apexlegends",
    "asoiaf",
    "Market76",
    "Animemes",
    "FortNiteBR",
    "nfl",
    "trashy",
    "unpopularopinion",
    "ChapoTrapHouse",
    "RoastMe",
    "Showerthoughts",
    "wallstreetbets",
    "Pikab",
]

In [194]:
subreddit_dict = {subreddit[i]:i for i in range(len(subreddit))}

# Sentence

In [9]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']

file_path = 'pre-processed-data.csv'
detected_encoding = detect_encoding(file_path)

df = pd.read_csv(file_path, encoding=detected_encoding)

In [10]:
df1 = df.copy()

In [11]:
df = df1.drop(columns=['score'])

df -->train and test
below --?train data 


In [13]:
from sklearn.model_selection import train_test_split

y = df['0']
X = df['body']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Train set shape: (800000,) (800000,)
Test set shape: (200001,) (200001,)


In [14]:
df_train = pd.concat([y_train,X_train], axis=1)

In [16]:
df_train.iloc[0]['body']

'love'

In [17]:
df_data = {'Subjectivity':[],'Polarity':[],'Neg':[],'Pos':[],'Compound':[],'Complexity':[],'Class':[]}

In [18]:
df_data

{'Subjectivity': [],
 'Polarity': [],
 'Neg': [],
 'Pos': [],
 'Compound': [],
 'Complexity': [],
 'Class': []}

In [19]:
def count_syllables(word):
    word = word.lower().strip()
    vowel_sounds = re.findall(r'[aeiouy]+', word)
    syllables = len(vowel_sounds)
    if word.endswith('e'):
        syllables -= 1
    if word.endswith('y') and not re.match(r'[aeiouy]+y$', word):
        syllables += 1
    return syllables

def flesch_kincaid_grade_level(text):
    words = text.split()
    sentences = re.split(r'[.?!]+', text.strip())

    avg_syllables = sum(count_syllables(word) for word in words) / len(words)
    avg_words_per_sentence = len(words) / len(sentences)

    fkgl = 0.39 * avg_words_per_sentence + 11.8 * avg_syllables - 15.59
    return round(fkgl, 2)

In [20]:
df_train.shape[0]

800000

In [21]:
df_train.iloc[0]


0         17
body    love
Name: 382311, dtype: object

In [22]:
for i in range(df_train.shape[0]):
    text = str(df_train.iloc[i]['body'])
    blob = TextBlob(text)
    df_data['Subjectivity'].append(blob.subjectivity)
    df_data['Polarity'].append(blob.sentiment.polarity)
    polarity_scores = onj.polarity_scores(text)
    df_data['Neg'].append(polarity_scores['neg'])
    df_data['Pos'].append(polarity_scores['pos'])
    df_data['Compound'].append(polarity_scores['compound'])
    fkgl = flesch_kincaid_grade_level(text)
    df_data['Complexity'].append(fkgl)
    df_data['Class'].append(df_train.iloc[i][0])

C:\Users\harsh\AppData\Local\Temp\ipykernel_11692\1005280258.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_data['Class'].append(df_train.iloc[i][0])


In [23]:
df_data_train = pd.DataFrame(df_data)

In [24]:
df_data_train.head()

,Subjectivity,Polarity,Neg,Pos,Compound,Complexity,Class
0,0.600000,0.50000,0.000,1.00,0.6369,-3.40,17
1,0.500000,0.50000,0.000,0.42,0.4404,2.88,19
2,0.000000,0.00000,0.000,0.00,0.0000,2.89,27
3,0.583333,-0.44375,0.233,0.30,0.0018,1.29,26
4,0.000000,0.00000,0.000,0.00,0.0000,7.60,3


In [25]:
df_data_train.to_csv('Mid_data_train.csv',index = False)

## Training

In [26]:
data = pd.read_csv("Mid_data_train.csv")

In [27]:
features = data.iloc[:,:6]
labels = data.iloc[:,-1]

In [28]:
class_medians = features.groupby(labels).mean()

In [29]:
def return_params(text):
    blob = TextBlob(text)
    polarity_scores = onj.polarity_scores(text)
    fkgl = flesch_kincaid_grade_level(text)
    return [blob.subjectivity,blob.sentiment.polarity,polarity_scores['neg'],polarity_scores['pos'],polarity_scores['compound'],fkgl]

In [30]:
def cosine_similarity(data_point1, data_point2):
    dot_product = np.dot(data_point1, data_point2)
    norm1 = np.linalg.norm(data_point1)
    norm2 = np.linalg.norm(data_point2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [31]:
df_test = pd.concat([y_test,X_test], axis=1)

In [32]:
df_test

,0,body
624589,32,pff example ferrell listed 35 said *ferrell ra...
79954,35,started work 99 boomer younger asshole vigor b...
567130,29,combination following aae pump action shotgun ...
500891,8,"compare pharma marketing budget r &amp;d, comp..."
55399,8,wasn 't illegitimate election
...,...,...
639297,31,kevin *thoroughly* disliked cousin
311939,24,"way saw wa wa waiting climb drogon ""bran"" like..."
324459,21,friend mentioned idea black widow movie end cr...
390499,24,actually 15 probably 14 filming scene **bella ...


In [33]:
test_data_point = []
for i in range(df_test.shape[0]):
    test_text = str(df_test.iloc[i]['body'])
    test_data_point.append(return_params(test_text))

In [36]:
list_sim = []
for j in range(len(test_data_point)):
    similarity = {}
    for i in range(class_medians.shape[0]):
        similarity[i] = cosine_similarity(test_data_point[j],list(class_medians.iloc[i]))
    list_sim.append(similarity)

C:\Users\harsh\AppData\Local\Temp\ipykernel_11692\1848760392.py:5: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (norm1 * norm2)


In [98]:
top_elements = []
for i in range(len(list_sim)):
    top_elements.append(list(list_sim[i].items()))

In [100]:
top_elements[0]

[(0, 0.9945848426909374),
 (1, 0.9899895007532584),
 (2, 0.9925434243530014),
 (3, 0.9951104386596537),
 (4, 0.9941171739023634),
 (5, 0.9919994519140726),
 (6, 0.9926167730479974),
 (7, 0.9932586442083119),
 (8, 0.9940587855846826),
 (9, 0.9935647040526497),
 (10, 0.9936378807484312),
 (11, 0.9928481613228304),
 (12, 0.994709058112989),
 (13, 0.9924803800988937),
 (14, 0.9936549435166017),
 (15, 0.9936677353342029),
 (16, 0.9927694044256955),
 (17, 0.9729587787852),
 (18, 0.9932738190536943),
 (19, 0.9935304505263797),
 (20, 0.9934150610230558),
 (21, 0.9930970213074652),
 (22, 0.9924921926175797),
 (23, 0.9931700503767211),
 (24, 0.9951157173867653),
 (25, 0.9932172594809789),
 (26, 0.9937413853964228),
 (27, 0.9936769849473188),
 (28, 0.9946563662497602),
 (29, 0.9894401508382134),
 (30, 0.9933763250914242),
 (31, 0.9930518993719555),
 (32, 0.9922989703652496),
 (33, 0.9947311079433254),
 (34, 0.9938438066400118),
 (35, 0.9942995496879816),
 (36, 0.9935436149483418),
 (37, 0.9928170

In [89]:
"""for i in range(len(top_elements)):
    max = top_elements[i][0][1]
    min = top_elements[i][-1][1]
    for j in range(len(top_elements[i])):
        top_elements[i][j] = (top_elements[i][j][0],(top_elements[i][j][1]-min)/(max-min))"""

# Word parameters

## Ishaan's code

In [101]:
token_dict={}

with open("pre-processed-data_`.csv", "r", encoding="utf8", errors="ignore") as f:

    k=f.readline()

    while(True):

        k=f.readline()
        
        if len(k)==0:
            break

        _,x,k=k.split(",",2)

        k=re.sub('[\d|\_]', '', k)

        token_dict[x]=k

In [102]:
tfidf = TfidfVectorizer(sublinear_tf=True, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

feature_names = tfidf.get_feature_names_out()

dense=tfs.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names, index= list(token_dict.keys()))

In [104]:
subreddit_lists = {i:subreddit[i] for i in range(len(subreddit))}

In [106]:
list_indices = df.index

In [107]:
list_indices = {i:i for i in list_indices}

In [196]:
list_indices['Pikabu'] = 'Pikab'

In [197]:
df = df.rename(index=list_indices)

In [250]:
def count_matching_words(text,iter,top_elements):
  text = re.sub('[\d|\_]','',text)
  text = re.sub('[\(|\)|/]',' ',text)
  out={}
  for i in df.index:
    out[i]=top_elements[iter][subreddit_dict[i]][1]**2
    
  for word in text.split():

    if word not in df.columns:
      continue
      
    for sub in df.index:

      if df.loc[sub][word]>0:
        out[sub]=out[sub]+(df.loc[sub][word])**0.5

  out=dict(sorted(out.items(), reverse=True, key=lambda item:item[1]))
  return list(out.keys())[0]
  

In [ ]:
output_predict_lists = []
acc=0
for i in X_test.index:
    output_predict_lists.append(count_matching_words(str(X_test[i]),k,top_elements))
    k+=1

    if subreddit_lists[y_test[i]]==output_predict_lists[-1]:
        acc+=1

acc

In [255]:
acc

80203

In [257]:
print("Accuracy of the model is: ",acc/len(X_test)*100)

Accuracy of the model is:  40.10129949350254


# You dont need to add the next part, it is for further extensions

In [ ]:
import pandas as pd
from textblob import TextBlob
import re


def analyze_text(text):
    """
    Analyzes a text and returns a dictionary containing values for 6 dimensions:
        - joy
        - anger
        - complexity (average number of syllables per word)
    """
    # Create a TextBlob object
    blob = TextBlob(text)

    # Calculate emotion scores (range: 0-1)
    joy = blob.sentiment.polarity
    anger = blob.sentiment.subjectivity

    # Calculate word complexity (average syllables per word)
    syllables = sum(count_syllables(word) for word in text.split())
    word_count = len(text.split())
    complexity = syllables / word_count if word_count else 0


    # Return dictionary with analysis results
    return {
        "joy": joy,
        "anger": anger,
        "complexity": complexity,
    }


def count_syllables(word):
    # Remove punctuation and convert to lowercase
    word = word.lower().strip()

    # Count vowel sounds
    vowel_sounds = re.findall(r"[aeiouy]+", word)

    # Count syllables (assume consonant sounds between vowel sounds)
    syllables = len(vowel_sounds)

    # Special cases for silent "e" and "y"
    if word.endswith("e"):
        syllables -= 1
    if word.endswith("y") and not re.match(r"[aeiouy]+y$", word):
        syllables += 1

    return syllables


def flesch_kincaid_grade_level(text):

    words = text.split()
    sentences = re.split(r"[.?!]+", text.strip())

    avg_syllables = sum(count_syllables(word) for word in words) / len(words)
    avg_words_per_sentence = len(words) / len(sentences)

    fkgl = 0.39 * avg_words_per_sentence + 11.8 * avg_syllables - 15.59

    return round(fkgl, 2)

text = "Supercalifragilisticexpialidocious"
analysis_data = analyze_text(text)

print(analysis_data)
